Meine Ablage/
└── E-Mod/
    └── UE1-Daten/
    ├── Ergebnisse/
    │   └── Y_load_values.csv


In [ ]:
# manuelles Hochladen von bestimmten eigens ausgewählten Dateien (Aus lokalem Speicher)
# Speicherort: im Arbeitsverzeichnis unter contetnt
# Verwendungsbeispiel: df = pd.read_csv("example.csv")
# from google.colab import files
# uploaded = files.upload()


from google.colab import drive
drive.mount('/content/drive')

# Absoluter Pfad zu deinem Datenordner in Google Drive
folder_path = "/content/drive/MyDrive/E-Mod/UE1-Daten/"

# Ergebnisse speichern in
results_path = "/content/drive/MyDrive/E-Mod/Ergebnisse/"
os.makedirs(results_path, exist_ok=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Verknüpfen von persönlicher Google Drive mit Notebook
#from google.colab import drive
#drive.mount('/content/drive')

# (A) Preiselastizitätsmodell

## (A1)
Welche Preiselastizität ist in den vorhandenen Daten erkennbar? Ist diese signifikant?

$$
  E = C p^\alpha
$$

## (A2)

Setze die Stromnachfrage in Zusammenhang mit dem Strompreis für alle vorhandenen stündlichen Datenpunkte und bestimme $\alpha$.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import os

# ---0. Timestamps
timestamps = pd.date_range(start="2023-01-01 00:00", periods=8760, freq="h")

# === 1. Daten laden ===
# Ersetze 'nachfrage.csv' und 'preise.csv' durch deine tatsächlichen Dateinamen
#folder_path = "./Daten_UE1/"

# Dependend Variable - Y
hourly_load_profile = "hourly_load_profile_electricity_AT_2023.csv"

########################## Y richtig einlesen
# 1. Datei einlesen (ohne Header)

#df_raw = pd.read_csv(os.path.join(folder_path, hourly_load_profile), sep=";")
df_raw = pd.read_csv(os.path.join(hourly_load_profile), sep=";")

# Timestamp aus DateUTC extrahieren (in Datetime umwandeln)
df_raw["timestamp"] = pd.to_datetime(df_raw["DateUTC"], format="%d-%m-%Y %H:%M")

# Relevante Spalten extrahieren: Timestamp + Verbrauch
Y = df_raw[["timestamp", "Value"]].rename(columns={"Value": "load"})

# Index zurücksetzen (falls nötig)
Y.reset_index(drop=True, inplace=True)

# Optional abspeichern
Y.to_csv(os.path.join(results_path, "Y_load_values.csv"), index=False)

########################## Preise richtig einlesen

# --- Einlesen: Preise
#df_price = pd.read_csv(os.path.join(folder_path, "preise2023.csv"))
df_price = pd.read_csv(os.path.join("preise2023.csv"))
df_price = df_price.rename(columns={"AT": "Price"})
df_price["timestamp"] = timestamps

####################################################


df_demand = Y

# df_demand["timestamp"] = pd.to_datetime(df_demand["timestamp"]) # Anpassen der Timestamps um Mergen zu ermöglichen
# df_price["timestamp"] = pd.to_datetime(df_price["timestamp"]) # Anpassen der Timestamps um Mergen zu ermöglichen
df = pd.merge(df_demand, df_price, on="timestamp")

# === 2. Einheiten anpassen ===
# Preis von ct/kWh in €/MWh (10/1000 ct/kWh = €/MWh)
df["price_eur_mwh"] = df["Price"] * 100/1000

# === 3. Logarithmieren ===

# Sicherstellen, dass 'load' numerisch ist
df["load"] = pd.to_numeric(df["load"], errors="coerce")

# Nur positive Werte behalten (log() ist nur für > 0 definiert)
df = df[(df["load"] > 0) & (df["price_eur_mwh"] > 0)]

df["log_price"] = np.log(df["price_eur_mwh"])
df["log_demand"] = np.log(df["load"])

# === 4. Regression ===
X = sm.add_constant(df["log_price"])  # (ln(C)) +log Strompreis
y = df["log_demand"] # logarithmische Energienachfrage (y-Achse)

model = sm.OLS(y, X).fit()
print(model.summary())

# === 5. Plot der Regression ===
plt.figure(figsize=(8, 5))
plt.scatter(df["log_price"], df["log_demand"], alpha=0.3, label="Daten") # alpha 30% sichtbar (optische Darstellung)
plt.plot(df["log_price"], model.predict(X), color="red", label="Regression")
plt.xlabel("log(Preis in €/MWh)")
plt.ylabel("log(Nachfrage in MWh/h)")
plt.title("Log-Log-Regression: Stromnachfrage vs. Preis")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

print(model.pvalues)

# === 6. Ergebnis interpretieren ===
alpha = model.params["log_price"]
p_value = model.pvalues["log_price"]


print(f"\nGeschätzte Preiselastizität (α): {alpha:.4f}")
print(f"p-Wert: {p_value:.4f}")
if p_value < 0.05:
    print("→ Die Preiselastizität ist signifikant auf dem 5%-Niveau.")
else:
    print("→ Die Preiselastizität ist NICHT signifikant.")

FileNotFoundError: [Errno 2] No such file or directory: 'hourly_load_profile_electricity_AT_2023.csv'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# (B) Nachfragemodellierung

Hierzu sind vorhandene Daten (z.B. Tagesstunde, Jahreszeit, Strompreis) zu
verwenden aber auch andere Open-Source-Daten. Formuliere zwei allgemeine
Modelle, wofür du mindestens vier unabhängige Variablen verwendest.
Mindestens ein Datensatz soll basierend auf deiner Recherche hinzugefügt
werden.

## (B1)

Stelle mindestens zwei Modell (=mind 2 unterschiedliche unabh. Variablen) auf
mit klarer Erklärung zur Auswahl der unabhängigen Variablen (weshalb sollten
diese signifikant sein?)

## (B2)

Bestimme Koeffizienten mittels Python/MATLAB/Excel...

HINWEISE: FRAGEN

Wurde die Einheitenrechnung Preis c/kWh und Nachfrage in MWh/h beachtet?

In [ ]:
from google.colab import files
import statsmodels.api as sm
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt


# uploaded = files.upload()

# Strings
# folder_path = "/content/drive/My Drive/E-MOD/Daten_UE1"
folder_path = "./Daten_UE1/"



# Y
# Hourly load profile
# hourly_load_profile_csv = "hourly_load_profile_electricity_AT_2023.csv"
# hourly_load_profile_csv_df = pd.read_csv(os.path.join(folder_path, hourly_load_profile_csv))


#############################################

# ---0. Timestamps
timestamps = pd.date_range(start="2023-01-01 00:00", periods=8760, freq="h")

# --- 1. Temperature - C°
hourly_temperature_profile_csv = "Messstationen Stundendaten v2 Datensatz_20230101T0000_20231231T2300.csv"
df_temp = pd.read_csv(os.path.join(folder_path, hourly_temperature_profile_csv), parse_dates=["time"])

# Delete Station-name and timestamps - timstamps will be generated (timestamps). This is because using these timestamps and generated ones created a merge problem
df_temp = df_temp.drop(columns=["station"])
df_temp = df_temp.drop(columns=["time"])

df_temp["timestamp"] = timestamps

df_temp = df_temp.rename(columns={"tl": "Temperature"})

X1 = df_temp

# ---3. Weekends
X1["Weekend"] = X1["timestamp"].dt.weekday >= 5
X1["Weekend"] = X1["Weekend"].astype(int)

# Spaltenreihenfolge ändern
X1 = X1[["timestamp"] + [col for col in X1.columns if col != "timestamp"]]

X1.to_csv("./Ergebnisse/x1_variables.csv", index=False)


# Dependend Variable - Y
hourly_load_profile = "hourly_load_profile_electricity_AT_2023.csv"


# 1. Datei einlesen (ohne Header)
# CSV einlesen
df_raw = pd.read_csv(os.path.join(folder_path, hourly_load_profile), sep=";")

# Timestamp aus DateUTC extrahieren (in Datetime umwandeln)
df_raw["timestamp"] = pd.to_datetime(df_raw["DateUTC"], format="%d-%m-%Y %H:%M")

# Relevante Spalten extrahieren: Timestamp + Verbrauch
Y = df_raw[["timestamp", "Value"]].rename(columns={"Value": "load"})

# Index zurücksetzen (falls nötig)
Y.reset_index(drop=True, inplace=True)

# Optional abspeichern
Y.to_csv("./Ergebnisse/Y_load_values.csv", index=False)


### Modeling

# 1. Addiere den Konstanten-Term (Intercept)
X1_const = sm.add_constant(X1.drop(columns=["timestamp"]))  # timestamp nicht mit reinnehmen

# Komma durch Punkt ersetzen und in float umwandeln
Y["load"] = Y["load"].str.replace(",", ".", regex=False).astype(float)

# 2. Modell schätzen
model = sm.OLS(Y["load"], X1_const).fit()

# Print the summary statistics

print("This is the first model")
print(model.summary())

# --- Vorhersagedaten vorbereiten
temp_range = np.linspace(X1["Temperature"].min(), X1["Temperature"].max(), 100)

model_features = X1_const.columns  # ['const', 'Temperature', 'Weekend']

# --- Linie für Werktage
df_line_weekday = pd.DataFrame({
    "const": 1,
    "Temperature": temp_range,
    "Weekend": 0
})

# --- Linie für Wochenende
df_line_weekend = pd.DataFrame({
    "const": 1,
    "Temperature": temp_range,
    "Weekend": 1
})

# --- Vorhersage berechnen
y_line_weekday = model.predict(df_line_weekday)
y_line_weekend = model.predict(df_line_weekend)

# --- Plot erstellen
plt.figure(figsize=(10, 6))

# Streudiagramm der echten Daten
colors = X1["Weekend"].map({0: "tab:blue", 1: "tab:orange"})
plt.scatter(X1["Temperature"], Y["load"], c=colors, alpha=0.5, label="Daten")

# Regressionslinien
plt.plot(temp_range, y_line_weekday, color="blue", linewidth=2, label="Regression (Werktag)")
plt.plot(temp_range, y_line_weekend, color="orange", linewidth=2, label="Regression (Wochenende)")

# Plot-Details
plt.xlabel("Temperatur (°C)")
plt.ylabel("Stromverbrauch (MWh/h)")
plt.title("Stromverbrauch vs. Temperatur (Werktage vs. Wochenende)")
plt.legend()
plt.grid(True)
plt.tight_layout()

# Abspeichern
# Sicherstellen, dass der Ordner existiert
os.makedirs("./Bilder", exist_ok=True)

# Plot speichern
plt.savefig("./Bilder/modell1_plot.png", dpi=300, bbox_inches="tight")

plt.show()



FileNotFoundError: [Errno 2] No such file or directory: './Daten_UE1/Messstationen Stundendaten v2 Datensatz_20230101T0000_20231231T2300.csv'

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import os

# --- Helper für Jahreszeiten
def get_season(month):
    if month in [3, 4, 5]:
        return 1  # Frühling
    elif month in [6, 7, 8]:
        return 2  # Sommer
    elif month in [9, 10, 11]:
        return 3  # Herbst
    else:
        return 4  # Winter

# --- Pfad
folder_path = "./Daten_UE1/"

# --- Zeitstempel für 2023
timestamps = pd.date_range(start="2023-01-01 00:00", periods=8760, freq="h")

# --- Einlesen: Preise
df_price = pd.read_csv(os.path.join(folder_path, "preise2023.csv"))
df_price = df_price.rename(columns={"AT": "Price"})
df_price["timestamp"] = timestamps
df_price["Season"] = df_price["timestamp"].dt.month.apply(get_season)

# --- Neue Gruppierung: Winter & Herbst = 1, Sommer & Frühling = 0
df_price["ColdSeason"] = df_price["Season"].apply(lambda s: 1 if s in [3, 4] else 0)

# --- Einlesen: Last (Y)
df_raw = pd.read_csv(os.path.join(folder_path, "hourly_load_profile_electricity_AT_2023.csv"), sep=";")
df_raw["timestamp"] = pd.to_datetime(df_raw["DateUTC"], format="%d-%m-%Y %H:%M")
Y2 = df_raw[["timestamp", "Value"]].rename(columns={"Value": "load"})
Y2["load"] = Y2["load"].str.replace(",", ".", regex=False).astype(float)
Y2.reset_index(drop=True, inplace=True)

# --- X definieren
X2 = df_price[["timestamp", "Price", "ColdSeason"]]

# --- Preise kleiner oder gleich 0 ausschließen
X2 = X2[X2["Price"] > 0].reset_index(drop=True)
Y2 = Y2.loc[X2.index].reset_index(drop=True)

# --- Konstante hinzufügen
X2_const = sm.add_constant(X2.drop(columns=["timestamp"]))

# --- Modell schätzen
model2 = sm.OLS(Y2["load"], X2_const).fit()

# --- Ergebnisse anzeigen
print("Modell 2 – Stromverbrauch ~ Preis + ColdSeason")
print(model2.summary())

# --- Vorhersagedaten für Plot
price_range = np.linspace(X2["Price"].min(), X2["Price"].max(), 100)

# --- Farben definieren
cold_colors = {0: "tab:green", 1: "tab:blue"}
point_colors = X2["ColdSeason"].map(cold_colors)

# --- Plot
plt.figure(figsize=(10, 6))

# Linie: Frühling/Sommer
df_line_summer = pd.DataFrame({
    "const": 1,
    "Price": price_range,
    "ColdSeason": 0
})
plt.plot(price_range, model2.predict(df_line_summer), label="Frühling/Sommer", color="tab:green")

# Linie: Herbst/Winter
df_line_winter = pd.DataFrame({
    "const": 1,
    "Price": price_range,
    "ColdSeason": 1
})
plt.plot(price_range, model2.predict(df_line_winter), label="Herbst/Winter", color="tab:blue")

# Punkte
plt.scatter(X2["Price"], Y2["load"], c=point_colors, alpha=0.5, label="Daten")

# Layout
plt.xlabel("Strompreis (ct/kWh)")
plt.ylabel("Stromverbrauch (MWh/h)")
plt.title("Stromverbrauch vs. Strompreis – Kalte vs. Warme Jahreszeiten")
plt.legend()
plt.grid(True)
plt.tight_layout()

# Abspeichern
# Sicherstellen, dass der Ordner existiert
os.makedirs("./Bilder", exist_ok=True)

# Plot speichern
plt.savefig("./Bilder/modell2_plot.png", dpi=300, bbox_inches="tight")


plt.show()

## (B3)

Interpretiere die Ergebnisse

# (C) Strompreismodellierung

Fragestellung: Welche unabhängigen Variablen beeinflussen die Strompreis?
Hierzu sind vorhandene Daten (z.B. Tagesstunde, Jahreszeit, Nachfrage) zu
verwenden aber auch andere Open-Source-Daten. Formuliere zwei allgemeine
Modelle, wofür du mindestens vier unabhängige Variablen verwendest.
Mindestens ein Datensatz soll basierend auf deiner Recherche hinzugefügt
werden.

## (C1)

Stelle mindestens Modelle (=mind 2 unterschiedliche unabh. Variablen) auf mit
klarer Erklärung zur Auswahl der unabhängigen Variablen (weshalb sollten
diese signifikant sein?)

## (C2)

Bestimme Koeffizienten mittels Python/MATLAB/Excel...

## (C3)

Interpretiere die Ergebnisse